In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_excel("/content/Churn_Modelling (1).xlsx")

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0
1,2.0,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0
2,3.0,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0
3,4.0,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0
4,5.0,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0


In [ ]:
!pip install -q keras

In [ ]:
x=df.iloc[:,3:13]
y=df.iloc[:,13]

In [ ]:
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619.0,France,Female,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,502.0,France,Female,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,699.0,France,Female,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771.0,France,Male,39.0,5.0,0.00,2.0,1.0,0.0,96270.64
9996,516.0,France,Male,35.0,10.0,57369.61,1.0,1.0,1.0,101699.77
9997,709.0,France,Female,36.0,7.0,0.00,1.0,0.0,1.0,42085.58
9998,772.0,Germany,Male,42.0,3.0,75075.31,2.0,1.0,0.0,92888.52


In [ ]:
y

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    1.0
9998    1.0
9999    0.0
Name: Exited, Length: 10000, dtype: float64

In [ ]:
# creating dummy variables
geography=pd.get_dummies(x['Geography'],drop_first=True)
gender=pd.get_dummies(x['Gender'],drop_first=True)

In [ ]:
geography

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,0


In [ ]:
gender

,Male
0,0
1,0
2,0
3,0
4,0
...,...
9995,1
9996,1
9997,0
9998,1


In [ ]:
# Concatenate the dataframe
x=pd.concat([x,geography,gender],axis=1)

In [ ]:
# Drop the unnecessary column
x=x.drop(['Geography','Gender'],axis=1)

In [ ]:
# Splitting data into train and test set
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.fit_transform(xtest)


In [ ]:
# Make the ANN
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
# Initialize the ann
classifier=Sequential()


In [ ]:
# Adding input layer and hidden layer
classifier.add(Dense(units=10,kernel_initializer='he_normal',activation='relu',input_dim=11))
classifier.add(Dropout(0.3))

# Adding second hidden layer
classifier.add(Dense(units=20,kernel_initializer='he_normal',activation='relu'))
classifier.add(Dropout(0.4))

# Adding third hidden layer
classifier.add(Dense(units=15,kernel_initializer='he_normal',activation='relu'))
classifier.add(Dropout(0.2))

# adding the output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

# Compiling the ann
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# fitting the ann to training set
model_history=classifier.fit(xtrain,ytrain,validation_split=0.33,batch_size=10,epochs=100)

# Predicting the test result
ypred=classifier.predict(xtest)
ypred=(ypred>0.5)

# Making confusion metrics
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(ytest,ypred)
print(cm)

# Calculate the accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(ypred,ytest)
print(score)

Epoch 1/100
536/536 [==============================] - 4s 6ms/step - loss: 0.5659 - accuracy: 0.7837 - val_loss: 0.5087 - val_accuracy: 0.7955
Epoch 2/100
536/536 [==============================] - 3s 6ms/step - loss: 0.5162 - accuracy: 0.7962 - val_loss: 0.5064 - val_accuracy: 0.7955
Epoch 3/100
536/536 [==============================] - 3s 6ms/step - loss: 0.5041 - accuracy: 0.7966 - val_loss: 0.4550 - val_accuracy: 0.7955
Epoch 4/100
536/536 [==============================] - 3s 6ms/step - loss: 0.4385 - accuracy: 0.8143 - val_loss: 0.4109 - val_accuracy: 0.8167
Epoch 5/100
536/536 [==============================] - 3s 6ms/step - loss: 0.4228 - accuracy: 0.8242 - val_loss: 0.3978 - val_accuracy: 0.8338
Epoch 6/100
536/536 [==============================] - 3s 6ms/step - loss: 0.4268 - accuracy: 0.8272 - val_loss: 0.4095 - val_accuracy: 0.7955
Epoch 7/100
536/536 [==============================] - 3s 6ms/step - loss: 0.4141 - accuracy: 0.8306 - val_loss: 0.4020 - val_accuracy: 0.8220